### 1. Vector Store Retriver


In [2]:
%pip install -q langchain_openai langchain_community langchain langchain_text_splitters

Note: you may need to restart the kernel to use updated packages.


# My Model


# WIP


In [4]:
from langchain_community.document_loaders.csv_loader import CSVLoader


from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.vectorstores.utils import DistanceStrategy
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain.embeddings import CacheBackedEmbeddings
from langchain.storage import LocalFileStore

In [6]:
# csv load
loader = CSVLoader(file_path="ss_data.csv")
data = loader.load()

RuntimeError: Error loading ss_data.csv

In [ ]:

#1.문서로드
directory_path = 'cx_categories'  # 텍스트 파일이 있는 디렉토리 경로를 지정하세요.
loader = DirectoryLoader(directory_path, glob='**/*.txt')
documents = loader.load()


# 3. 캐시 디렉토리설정: cache 폴더를 생성하고, 임베딩 데이터를 캐싱하기 위한 디렉토리로 설정
cache_dir = LocalFileStore("./.cache/")

# 4. 한국어 특화 모델로 문서 임베더 로드
embeddings_model = HuggingFaceEmbeddings(
    model_name='jhgan/ko-sbert-nli',
    model_kwargs={'device':'cpu'},
    encode_kwargs={'normalize_embeddings':True},
)

# 5. 캐시된 임베딩 설정: CacheBackedEmbeddings를 사용하여 임베딩 결과를 로컬 캐시에 저장
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings_model,
    cache_dir,
)

vectorStore = FAISS.from_documents(documents, embeddings_model)
retriever = vectorStore.as_retriever()



prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Answer questions using only the \
        following context. If you don't know the answer, just say you don't know\
        don't make it up: \n\n{context}"),
    ("human", "{question}")
])

# 1. retriever will return documents
# 2. documents will be inserted into template
chain = {
    "context": retriever,
    "question": RunnablePassthrough()
    }| prompt | llm

chain.invoke("Describe Victory Mansions") # RunnablePassth


In [ ]:
from langchain.chains import RetrievalChain
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import DirectoryLoader
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI

import os

# 1단계: 카테고리 정의 로드
category_loader = DirectoryLoader(
    directory="cx_categories",  # 카테고리 파일이 있는 디렉토리 경로
    loader="text",
    text_splitter=CharacterTextSplitter(separator="\n\n", chunk_size=200, chunk_overlap=50)
)

# 2단계: 임베딩 준비
embeddings = OpenAIEmbeddings()

# 3단계: 카테고리 정의를 위한 벡터 저장소 생성
category_store = Chroma.from_documents(category_loader.load(), embeddings)

# 4단계: Retrieval-Augmented Generation 체인 설정
llm = OpenAI(model_name="gpt-3.5-turbo", openai_api_key=os.getenv("OPENAI_API_KEY"))

prompt_template = """
주어진 리뷰 텍스트를 미리 정의된 카테고리 중 하나로 분류하세요. 카테고리는 제공된 문서로 정의됩니다. 정의를 사용하여 리뷰에 가장 적합한 카테고리를 결정하세요. 카테고리 이름과 간단한 이유를 제공하세요.

리뷰: {review}

카테고리 정의:
{context}

가장 적합한 카테고리와 선택한 이유를 출력하세요.
"""

prompt = PromptTemplate(
    input_variables=["review", "context"],
    template=prompt_template,
)

chain = RetrievalChain.from_llm(
    llm=llm,
    retriever=category_store.as_retriever(),
    prompt=prompt,
)

# 5단계: 리뷰를 카테고리에 맵핑하는 함수 정의
def map_review_to_category(review):
    result = chain.run({"review": review})
    return result

# 예시 사용:
reviews = [
    "제품 품질이 뛰어나고 고객 서비스가 신속했습니다.",
    "배송이 지연되고 포장이 손상되었습니다.",
    # 리뷰를 추가하세요
]

# 각 리뷰를 가장 적합한 카테고리로 맵핑
for review in reviews:
    category_mapping = map_review_to_category(review)
    print(f"리뷰: {review}\n카테고리 맵핑: {category_mapping}\n")
